In [91]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1433,tt0401855,50000000,111340801,Underworld: Evolution,Kate Beckinsale|Scott Speedman|Tony Curran|Sha...,Len Wiseman,"My God. Brother, what have you done?",As the war between the vampires and the Lycans...,106,Fantasy|Action|Science Fiction|Thriller,Lakeshore Entertainment|Screen Gems,1/12/2006,6.3,2006
1160,tt0289992,50000000,38955598,The Life of David Gale,Kevin Spacey|Kate Winslet|Laura Linney|Gabriel...,Alan Parker,The crime is clear. The truth is not.,A man against capital punishment is accused of...,130,Drama|Thriller|Crime,Universal Pictures|Intermedia Films|Saturn Films,2/21/2003,7.2,2003
60,tt3076658,35000000,137935567,Creed,Michael B. Jordan|Sylvester Stallone|Graham Mc...,Ryan Coogler,Your legacy is more than a name,The former World Heavyweight Champion Rocky Ba...,133,Drama,New Line Cinema|Warner Bros.|Metro-Goldwyn-May...,11/25/2015,7.3,2015
1237,tt2387559,26000000,51164106,Delivery Man,Vince Vaughn|Cobie Smulders|Chris Pratt|Britt ...,Ken Scott,You're never quite ready for what life delivers.,An affable underachiever finds out he's father...,105,Comedy,DreamWorks SKG|Touchstone Pictures,10/10/2013,6.1,2013
1440,tt0317919,150000000,397850012,Mission: Impossible III,Tom Cruise|Philip Seymour Hoffman|Ving Rhames|...,J.J. Abrams,The Mission Begins 05:05:06.,Retired from active duty to train new IMF agen...,126,Adventure|Action|Thriller,Paramount Pictures|Cruise/Wagner Productions|S...,5/3/2006,6.3,2006


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data['release_date'] = pd.to_datetime(data['release_date'])

# получение столбца profit:
data['profit'] = data['revenue'] - data['budget']

# функция возвращающая список из столбца с неоднозначным заполнением

def return_list(obj):
    s = ''
    for item in obj:
        s = s + item + '|'
    s = s[:len(s)-1]
    return s.split('|')


# создаем новый столбец, который состоит из жанров разделенных "|" для понимания начала и конца записи
data['own_genres'] = data['genres'].apply(lambda x: '|' + x + '|')

# создаем новый столбец, который состоит из режиссеров разделенных "|" для понимания начала и конца записи
data['own_director'] = data['director'].apply(lambda x: '|' + x + '|')

# создаем новый столбец, который состоит из актеров разделенных "|" для понимания начала и конца записи
data['own_cast'] = data['cast'].apply(lambda x: '|' + x + '|')

# создаем новый столбец, который состоит из студий разделенных "|" для понимания начала и конца записи
data['own_production_companies'] = data['production_companies'].apply(lambda x: '|' + x + '|')

In [6]:
data

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,own_genres,own_director,own_cast,own_production_companies
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810,|Action|Adventure|Science Fiction|Thriller|,|Colin Trevorrow|,|Chris Pratt|Bryce Dallas Howard|Irrfan Khan|V...,|Universal Studios|Amblin Entertainment|Legend...
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354,|Action|Adventure|Science Fiction|Thriller|,|George Miller|,|Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Ni...,|Village Roadshow Pictures|Kennedy Miller Prod...
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201,|Adventure|Science Fiction|Thriller|,|Robert Schwentke|,|Shailene Woodley|Theo James|Kate Winslet|Anse...,|Summit Entertainment|Mandeville Films|Red Wag...
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015,1868178225,|Action|Adventure|Science Fiction|Fantasy|,|J.J. Abrams|,|Harrison Ford|Mark Hamill|Carrie Fisher|Adam ...,|Lucasfilm|Truenorth Productions|Bad Robot|
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015,1316249360,|Action|Crime|Thriller|,|James Wan|,|Vin Diesel|Paul Walker|Jason Statham|Michelle...,|Universal Pictures|Original Film|Media Rights...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,2000-07-13,6.6,2000,82299717,|Adventure|Action|Science Fiction|,|Bryan Singer|,|Patrick Stewart|Hugh Jackman|Ian McKellen|Hal...,|Twentieth Century Fox Film Corporation|Donner...
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,2000-10-27,6.4,2000,-8444012,|Horror|Family|Foreign|,|Uli Edel|,|Richard E. Grant|Jonathan Lipnicki|Jim Carter...,|New Line Cinema|
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,2000-06-30,4.0,2000,-40865180,|Adventure|Animation|Action|Comedy|Family|,|Des McAnuff|,|Rene Russo|Jason Alexander|Piper Perabo|Randy...,|Universal Pictures|Capella International|KC M...
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"
# +

In [8]:
# тут пишем ваш код для решения данного вопроса:
data[data['budget'] == data['budget'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [9]:
# можно добавлять разные варианты решения
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,own_genres,own_director,own_cast,own_production_companies
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000,|Adventure|Action|Fantasy|,|Rob Marshall|,|Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian ...,|Walt Disney Pictures|Jerry Bruckheimer Films|...


# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111)' 
# +

In [11]:
data[data['runtime'] == data['runtime'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


ВАРИАНТ 2

In [12]:
# ищем индекс строки, в котрой максимальное значение продолжительности (idxmax), выводим элемент с этим индексом
data.loc[data['runtime'].idxmax()]

imdb_id                                                             tt0279111
budget                                                               56000000
revenue                                                              12923936
original_title                                              Gods and Generals
cast                        Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...
director                                                    Ronald F. Maxwell
tagline                                   The nations heart was touched by...
overview                    The film centers mostly around the personal an...
runtime                                                                   214
genres                                                      Drama|History|War
production_companies                       Turner Pictures|Antietam Filmworks
release_date                                              2003-02-21 00:00:00
vote_average                                                    

# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'
# +

In [14]:
data[data['runtime'] == data['runtime'].min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [15]:
answers['4'] = 110
# +

In [16]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = 107
# +

In [18]:
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
answers['6'] = '239. Avatar (tt0499549)'
# +

In [20]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data['profit'] == data['profit'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [21]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'
# +

In [22]:
data[data['profit'] == data['profit'].min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
answers['8'] = 1478
# +

In [24]:
len(data[data['profit'] > 0])

1478

ВАРИАНТ 2

In [25]:
len(data[data['revenue'] > data['budget']])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [26]:
answers['9'] = '599. The Dark Knight (tt0468569)'
# +

In [27]:
# делаем выборку за 2008 год
# сортируем по кассовым сборам в порядке убывания
# выводим название и id верхней строки таблицы
data[data['release_year'] == 2008].sort_values('revenue', ascending = False).head(1)[['original_title', 'imdb_id']]

,original_title,imdb_id
599,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [28]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'
# +

In [29]:
# делаем выборку по годам между 2012 и 2014
# сортируем по прибыли
# выводим название и id первой верхней строки нового датафрейма (с номером индекса 0)
data[(data['release_year'] >= 2012) & (data['release_year'] <= 2014)].sort_values('profit').iloc[0][['original_title','imdb_id']]

original_title    The Lone Ranger
imdb_id                 tt1210819
Name: 1245, dtype: object

ВАРИАНТ 2

In [30]:
# то же самое, только вывод через head для более простой записи в answer
data[(data['release_year'] >= 2012) & (data['release_year'] <= 2014)].sort_values('profit').head(1)[['original_title','imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [31]:
answers['11'] = 'Drama'
# +

In [32]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# получаем список со всеми жанрами в столбце genres
# помещаем его в Counter и выводим тот жанр, который встречается чаще всего
Counter(return_list(data['genres'])).most_common(5)

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415)]

ВАРИАНТ 2

In [33]:
# Counter списка, полученного путем раздела сумморного стринга из всех жанров без последнего символа "|"
cnt = Counter(data['own_genres'].sum()[:-1].split('|'))
# удаляем из Counter строку с "пустышкой", которая появляется из-за двойного "|" на стыках записей
del cnt['']
# выводим наиболее часто встречающиеся
cnt.most_common(5)

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [34]:
answers['12'] = 'Drama'
# +

In [35]:
Counter(return_list(data['genres'][data['profit'] > 0])).most_common(1)

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [36]:
answers['13'] = 'Peter Jackson'
# +

In [37]:
# группируем датафрейм по режиссерам
# суммируем всю группу и сортируем по кассовым сборам
# индекс первой записи в новом датафрейм и есть исходный режиссер
data.groupby('director').sum().sort_values('revenue', ascending = False).head(1).index

Index(['Peter Jackson'], dtype='object', name='director')

ВАРИАНТ 2 - более правильный

In [38]:
# если предположить, что режиссеров может быть несколько у одного фильма
# более правильный способ:

# создаем копию DataFrame, чтобы не "портить" исходник

data_13 = data.copy()
# в столбце "режиссер" меняем текстовую информацию на список из режиссеров снимавших этот фильм
# если фильм снимал один режиссер, то это будет список из одного элемента
data_13['director'] = data_13.director.str.split('|')

# дублируем строки где не один режиссер с помощью explode, группируем получившийся датафрейм по режиссерам, 
# суммируем кассовые сборы по всем фильмам каждого режиссера, сортируем по убыванию, выводим первую строку
data_13.explode('director').groupby('director').revenue.sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
answers['14'] = 'Robert Rodriguez'
# +

In [40]:
# делаем выборку режиссеров фильмов, в графе жанры которых присутствует '|Action|' 
# '|' символ начала и конца записи, который необходимо экранировать, иначе интерпретатор воспринимает его как "или"
# далее стандартный подсчет кол-ва вхождений режиссеров в список из выборки
Counter(return_list(data[data['own_genres'].str.contains('\|Action\|')]['director'])).most_common(5)

[('Robert Rodriguez', 9),
 ('Michael Bay', 7),
 ('Paul W.S. Anderson', 7),
 ('Antoine Fuqua', 6),
 ('Ridley Scott', 6)]

ВАРИАНТ 2 - более правильный

In [41]:
# создаем копию DataFrame, чтобы не "портить" исходник
data_14 = data.copy()

In [42]:
# делаем список из жанров
data_14['genres'] = data_14['genres'].str.split('|')
# делаем список из режиссеров
data_14['director'] = data_14['director'].str.split('|')
# дублируем строки где не один режиссер и/или не один жанр с помощью explode
# data_14 = data_14.explode('director', 'genres') !!!!!!!!!!!! Почему-то поделить одновременно НЕ получилось
data_14 = data_14.explode('director')
data_14 = data_14.explode('genres')

In [43]:
# группируем по жанрам, считаем какие режиссеры сколько раз "попидались" по жанрам, берем индекс Action, выводим первую строку 
# (т.к. value_counts уже отсортировал в нужном порядке)
data_14.groupby('genres')['director'].value_counts()['Action'].head(1)

director
Robert Rodriguez    9
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [44]:
answers['15'] = 'Chris Hemsworth'
# +

In [45]:
# создаем временный срез датафрейма по 2012 году
tmp_data = data[data['release_year'] == 2012]
# для решения задачи понадопиться объект Series с индексами уникальными именами всех актеров снимавшихся в 2012
# для создания индексов Series используем list от Counter, который вызврощает все уникальные ключи
index_list = list(Counter(return_list(tmp_data['cast'])))
# создаем Series, заполняем нулями
tmp = pd.Series(0, index = index_list)
# заполняем Series значениями: суммируем все кассовые сборы если имя актера встречается в фильме
for item in tmp.index:
    tmp[item] = tmp_data[tmp_data['own_cast'].str.contains('\|' + item + '\|')]['revenue'].sum()
# сортируем получившийся Series и выводим верхнюю строку
tmp.sort_values(ascending = False).head(1)

Chris Hemsworth    2027450773
dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [46]:
answers['16'] = 'Matt Damon'
# +

In [47]:
# определяем какой из актеров встречается чаще всех в выборке, где бюджет выше среднего в исходной статистике
Counter(return_list(data[data['budget'] > data['budget'].mean()]['cast'])).most_common(5)

[('Matt Damon', 18),
 ('Adam Sandler', 17),
 ('Angelina Jolie', 16),
 ('Tom Cruise', 15),
 ('Samuel L. Jackson', 15)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [48]:
answers['17'] = 'Action'
# +

In [49]:
# определяем какой жанр чаще всего встречается в фильмах, где встречается |Nicolas Cage| в колонке актеров
Counter(return_list(data[data['own_cast'].str.contains('\|Nicolas Cage\|')]['genres'])).most_common(2)

[('Action', 17), ('Thriller', 15)]

# 18. Самый убыточный фильм от Paramount Pictures

In [50]:
answers['18'] = 'K-19: The Widowmaker'
# +

In [51]:
# делаем выборку по точному совпадению студии, сортируем по возрастанию прибыли, выводим название верхнего фильма
data[data['own_production_companies'].str.contains('\|Paramount Pictures\|')].sort_values('profit').head(1)['original_title']

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [52]:
answers['19'] = 2015
# +

In [53]:
# группируем данные по годам, складываем кассовые сборы, сортируем по убыванию
data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [54]:
answers['20'] = 2014
# +

In [55]:
# делаем выборку по фильмам, где есть студия содержащая в названии Warner Bros, далее стандартно
data[data['production_companies'].str.contains('Warner Bros')].groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [56]:
answers['21'] = 'сентябрь'
# +

In [57]:
# группируем выборку по месяцу с помощью метода dt.month, выбираем столбец, например, с id фильмов и считаем кол-во по группам
# сортируем по убыванию
data.groupby(data['release_date'].dt.month)['imdb_id'].count().sort_values(ascending=False)

release_date
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [58]:
answers['22'] = 450
# +

In [59]:
# определяем кол-во строк в выборке, где дата производства фильма включает один из летних месяцев (6, 7, 8)
len(data.query('release_date.dt.month in [6,7,8]'))

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [60]:
answers['23'] = 'Peter Jackson'
# +

In [61]:
# делаем выборку по зимним месяцам, берем из него колонку режиссеров, получаем из нее список, через Counter выводим самые частые
Counter(return_list(data.query('release_date.dt.month in [1,2,12]')['director'])).most_common(5)

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4)]

ВАРИАНТ 2 - строго говоря, более правильный вариант

In [62]:
# делаем копию оригинала
data_23 = data.copy()
# делаем из колонки режиссеров списки
data_23['director'] = data_23['director'].str.split('|')
# увеличиваем кол-во строк, таким образом чтобы режиссер был один
data_23 = data_23.explode('director')
# выводим 5 самых частых режиссеров попавших в веборку по датам производства зимой
Counter(return_list(data_23.query('release_date.dt.month in [1,2,12]')['director'])).most_common(5)

[('Peter Jackson', 7),
 ('Clint Eastwood', 6),
 ('Steven Soderbergh', 6),
 ('Shawn Levy', 4),
 ('Nancy Meyers', 4)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [63]:
answers['24'] = 'Four By Two Productions'
# +

In [64]:
# создаем Counter, который по сути хранит кол-во фильмов по студиям
num_of_films = Counter(return_list(data['production_companies']))
# создаем список уникальных студий
index_list = list(num_of_films)

In [65]:
# создаем Series с нулевыми значениями и индексами - уникальными студиями
tmp = pd.Series(0, index = index_list)

In [66]:
# цикл для подсчета средней длины названий по студиям
for item in tmp.index:
# для каждой студии получаем по ней выборку, берем названия, суммируем кол-во символов
# regex = False нужно чтобы отключить регулярные выражения и не маскировать '|'
    tmp[item] = data[data['own_production_companies'].str.contains('|' + item + '|', regex = False)]['original_title'].str.len().sum()
# делим получившееся на кол-во фильмов студии - получаем удельную среднюю длину символов
    tmp[item] = tmp[item]/num_of_films[item]

In [67]:
tmp.sort_values(ascending = False).head(5)

Four By Two Productions      83
Jim Henson Company, The      59
Dos Corazones                47
Polsky Films                 46
Museum Canada Productions    46
dtype: int64

ВАРИАНТ 2

In [68]:
data_24 = data.copy()
# делаем новый столбец равный длине названия
data_24['length_original_title'] = data_24['original_title'].str.len()
# делаем из столбца студий списки
data_24['production_companies'] = data_24['production_companies'].str.split('|')
# получаем датафрейм с индивидуальными студиями
data_24 = data_24.explode('production_companies')
# группируем по уникальным студиям, берем длины названий, усредняем, сортируем, выводим 5
data_24.groupby('production_companies')['length_original_title'].mean().sort_values(ascending = False).head(5)

production_companies
Four By Two Productions      83.0
Jim Henson Company, The      59.0
Dos Corazones                47.0
Polsky Films                 46.0
Museum Canada Productions    46.0
Name: length_original_title, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [69]:
answers['25'] = 'Midnight Picture Show'
# +

In [70]:
# цикл для подсчета среднего кол-ва слов в описании по студиям с использованием данных предыдущей задачи
for item in index_list:
# для каждой студии получаем по ней выборку, берем описания, делим на список по пробелам, суммируем весь список, определяем его длинну (кол-во слов)
# regex = False нужно чтобы отключить регулярные выражения и не маскировать '|'
    tmp[item] = len(data[data['own_production_companies'].str.contains('|' + item + '|', regex=False)]['overview'].str.split().sum())
# делим на кол-во фильмов, применяя counter из предыдущей задачи
    tmp[item] = tmp[item]/num_of_films[item]
tmp.sort_values(ascending = False).head(5)

Midnight Picture Show               175
Room 9 Entertainment                161
98 MPH Productions                  159
Heineken Branded Entertainment      159
Brookwell-McNamara Entertainment    156
dtype: int64

ВАРИАНТ 2

In [71]:
data_25 = data.copy()
# делаем новый столбец равный кол-ву слов в описании, сначала разделив на списки по пробелам, затем применив функцию len к каждому элементу
data_25['length_words_overview'] = data_25['overview'].str.split().apply(lambda x: len(x))
# делаем из столбца студий списки
data_25['production_companies'] = data_25['production_companies'].str.split('|')
# получаем датафрейм с индивидуальными студиями
data_25 = data_25.explode('production_companies')
# группируем по уникальным студиям, берем кол-во слов в описании, усредняем, сортируем, выводим 5
data_25.groupby('production_companies')['length_words_overview'].mean().sort_values(ascending = False).head(5)

production_companies
Midnight Picture Show               175.0
Room 9 Entertainment                161.0
Heineken Branded Entertainment      159.0
98 MPH Productions                  159.0
Brookwell-McNamara Entertainment    156.0
Name: length_words_overview, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [72]:
answers['26'] = ['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars',
 'The Lord of the Rings: The Two Towers']
# +

In [73]:
# сортируем по рейтингу, выводим 1% от общего кол-ва "сверху", оставляем только названия, вывод списком удобнее для чтения
list(data.sort_values('vote_average', ascending = False).head(round(len(data)*0.01))['original_title'])

['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars',
 'The Lord of the Rings: The Two Towers']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [74]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'
# +

In [75]:
# получаем список уникальных имен актеров
actors = list(Counter(return_list(data['cast'])))

In [76]:
# создаем пустой датафрейм
tmp = pd.DataFrame(0, index = actors, columns = ['partner', 'times'])
# делаем столбец partner текстовым
tmp.partner = tmp.partner.astype('string')
# проверяем
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3406 entries, Chris Pratt to Tess Harper
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   partner  3406 non-null   string
 1   times    3406 non-null   int64 
dtypes: int64(1), string(1)
memory usage: 79.8+ KB


In [77]:
# для каждого индивидуального актера
for item in actors:
# создаем Counter из всех актеров, встречающихся в фильмах в выборке с данным
    cnt = Counter(return_list(data[data['own_cast'].str.contains('\|' + item + '\|')]['cast']))
# удаляем запись про самого этого актера
    del cnt[item]
# с помощью мульти присвоения заполняем датафрейм самым часто встречающимся актером и кол-вом пересечений
    tmp.loc[item, 'partner'], tmp.loc[item, 'times'] = cnt.most_common(1)[0]

In [78]:
# сортируем по кол-ву появленийвместе и выводим
tmp.sort_values('times',ascending=False).head(10)

,partner,times
Emma Watson,Daniel Radcliffe,8
Rupert Grint,Daniel Radcliffe,8
Daniel Radcliffe,Rupert Grint,8
Johnny Depp,Helena Bonham Carter,6
Owen Wilson,Ben Stiller,6
Helena Bonham Carter,Johnny Depp,6
Ben Stiller,Owen Wilson,6
Vin Diesel,Paul Walker,5
Hugh Jackman,Ian McKellen,5
Ian McKellen,Hugh Jackman,5


ВАРИАНТ 2

In [109]:
# берем от исходных данных только колонку с актерами (Series)
tmp = data['cast']
# получаем списки из актеров, после применяем к ним combinations чтобы получить итератор из всех возможных сочетаний по два
# нам нужен из него список
tmp = tmp.str.split('|').apply(lambda a: list(combinations(a, 2)))
# суммируем всю колонку - получаем общий список, отправляем в counter, выводим самые частые
Counter(tmp.sum()).most_common(10)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5),
 (('Robert Pattinson', 'Taylor Lautner'), 5)]

# Submission

In [79]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her

In [80]:
# и убедиться что ни чего не пропустил)
len(answers)

27